In [8]:
import pyspark
from delta import *
import os
from pyspark.sql.functions import current_timestamp
from pyspark.sql import Row

In [9]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [10]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

25/04/29 19:11:50 WARN Utils: Your hostname, Alfios-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.50.4 instead (on interface en0)
25/04/29 19:11:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-27f92732-6c93-4835-8500-9acecc8b042c;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 85ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-27f92732-6c93-4835-8500-9acecc8b042c
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/2ms)
25/04/29 19:11:50 

In [13]:
landing_path = "../data/letstalk_landing_zone_bdma/delta_news/entertainment"
deltane = DeltaTable.forPath(spark, landing_path)
dfne = deltane.toDF()

In [90]:
new_data_df = dfne.filter(dfne.author.startswith("Ann"))

In [91]:
new_data_df.write.format("delta").mode("append").save(landing_path)

In [70]:
deltane.filter(deltane.filter(''))

AttributeError: 'DeltaTable' object has no attribute 'insert'

In [133]:
deltane.history()

25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921639145 which is greater than the next commit timestamp 1745921639145.
25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639501 which is greater than the next commit timestamp 1745921639239.
25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639502 which is greater than the next commit timestamp 1745921639185.
25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639603 which is greater than the next commit timestamp 1745921639436.
25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639604 which is greater than the next commit timestamp 1745921639330.
25/04/29 17:47:11 WARN DeltaHistoryManager: Found Delta commit 7 with a timestamp 1745921639605 which is greater than the next commit timestamp 1745921639185.
25/04/29 17:47:11 WARN DeltaHistoryManager: Fo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
14,2025-04-29 15:15:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,13,Serializable,false,"{numFiles -> 3, n...",NULL,Apache-Spark/3.5....
13,2025-04-29 15:13:...,NULL,NULL,SET TBLPROPERTIES,"{properties -> {""...",NULL,NULL,NULL,12,Serializable,true,{},NULL,Apache-Spark/3.5....
12,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,11,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
11,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,10,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
10,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,9,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
9,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,8,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
8,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,7,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
7,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,6,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
6,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,4,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....
5,2025-04-29 12:13:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,4,Serializable,true,"{numFiles -> 1, n...","{""inserted_rows"":...",Apache-Spark/3.5....


In [104]:
new_data = spark.read.format("delta") \
    .option("readChangeData", "true") \
    .option("startingVersion", 0) \
    .option("endingVersion", 14) \
    .load(path=landing_path) \
    .filter("_change_type = 'insert'")

In [66]:
dfne = dfne.dropDuplicates()

In [67]:
deltatu = DeltaTable.forPath(spark, "../data/letstalk_landing_zone_bdma/delta_tmdb/upcoming")
dftu = deltatu.toDF()

In [55]:
deltatu.history()

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2025-04-29 12:14:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,1,Serializable,true,"{numFiles -> 1, n...","{""perc_rows_inser...",Apache-Spark/3.5....
1,2025-04-29 12:14:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,0,Serializable,true,"{numFiles -> 1, n...","{""perc_rows_inser...",Apache-Spark/3.5....
0,2025-04-29 12:14:...,NULL,NULL,WRITE,"{mode -> Append, ...",NULL,NULL,NULL,NULL,Serializable,true,"{numFiles -> 1, n...","{""perc_rows_inser...",Apache-Spark/3.5....


In [23]:
dftu.groupby('id').count().orderBy('count', ascending=False).show()

+-------+-----+
|     id|count|
+-------+-----+
|1244944|    3|
|1126166|    3|
|1064486|    3|
| 995926|    3|
|1233575|    3|
|1212855|    3|
|1226406|    3|
|1241436|    3|
| 324544|    3|
|1353117|    3|
|1388366|    3|
|1380415|    2|
| 970450|    2|
| 575265|    2|
| 986056|    2|
|1181107|    2|
|1233413|    2|
|1232546|    2|
|1249289|    1|
|1249213|    1|
+-------+-----+
only showing top 20 rows



In [24]:
dftu.filter(dftu.id == 1244944)

adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,ingestion_time,begin_date,end_date
false,/3lEV4CoKoeT2cZ4f...,"[27, 9648]",1244944,en,The Woman in the ...,In the aftermath ...,172.9102,/n0WS2TsNcS6dtaZK...,2025-03-27,The Woman in the ...,false,6.27,37,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/3lEV4CoKoeT2cZ4f...,"[27, 9648]",1244944,en,The Woman in the ...,In the aftermath ...,157.7558,/n0WS2TsNcS6dtaZK...,2025-03-27,The Woman in the ...,false,6.0,97,2025-04-27 12:53:...,2025-04-30,2025-05-21
false,/3lEV4CoKoeT2cZ4f...,"[27, 9648]",1244944,en,The Woman in the ...,In the aftermath ...,262.5169,/n0WS2TsNcS6dtaZK...,2025-03-27,The Woman in the ...,false,6.0,89,2025-04-23 08:47:...,2025-04-30,2025-05-21


In [18]:
relative_path = "../data/letstalk_landing_zone_bdma/delta_news/entertainment"
absolute_path = os.path.abspath(relative_path)
inc = IncrementalLoad(spark)
inc.get_new_data(absolute_path, '../version')

Using CDF from version 13


25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921639145 which is greater than the next commit timestamp 1745921639145.
25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639501 which is greater than the next commit timestamp 1745921639239.
25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639502 which is greater than the next commit timestamp 1745921639185.
25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639603 which is greater than the next commit timestamp 1745921639436.
25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639604 which is greater than the next commit timestamp 1745921639330.
25/04/29 19:13:17 WARN DeltaHistoryManager: Found Delta commit 7 with a timestamp 1745921639605 which is greater than the next commit timestamp 1745921639185.
25/04/29 19:13:17 WARN DeltaHistoryManager: Fo

(+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+---------------+--------------------+
 |              author|             content|         description|         publishedAt|              source|               title|                 url|          urlToImage|_change_type|_commit_version|   _commit_timestamp|
 +--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+---------------+--------------------+
 |          Anna Logan|Did you know Gigi...|From Hollywood to...|2025-04-26T13:17:46Z|{NULL, countryliv...|11 Celebrities Se...|https://www.count...|https://hips.hear...|      insert|             14|2025-04-29 15:15:...|
 |Anneta Konstantin...|Guy Fieri may hav...|Guy Fieri may hav...|2025-04-16T08:01:00Z|{business-insider...|Guy Fieri

In [20]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

schema = StructType([
    StructField("source_table", StringType(), False),
    StructField("last_processed_version", LongType(), False),
    StructField("last_run_ts", TimestampType(), True)
])

empty_df = spark.createDataFrame([], schema)

base_path = "../data/letstalk_landing_zone_bdma"
control_table_path = "control_table"
final_path = os.path.join(base_path, control_table_path)

empty_df.write.format("delta").mode("overwrite").save(final_path)


In [17]:
def does_cdf_make_sense(delta_table, last_version):
    history_df = delta_table.history()
    for row in history_df.collect():
        if '"delta.enableChangeDataFeed":"true"' in row['operationParameters'].get("properties", ""):
            cdf_start_version = row["version"]
            break
    else:
        cdf_start_version = None

    if cdf_start_version is not None and last_version >= cdf_start_version:
        return True

    return False


def is_cdf_enabled(delta_table):
    properties = delta_table.detail().selectExpr("properties['delta.enableChangeDataFeed']").collect()
    return properties[0][0] == "true"


class IncrementalLoad:
    def __init__(self, spark, base_path, table_name):
        self.spark = spark
        self.base_path = base_path
        self.control_table_path = os.path.join(self.base_path, "control_table")
        self.table_name = table_name
        self.landing_path = os.path.join(self.base_path, self.table_name)


    def update_control_table(self, latest_version):
        new_row = spark.createDataFrame([
            Row(source_table=self.table_name,
                last_processed_version=latest_version,
                last_run_ts=current_timestamp())
        ])

        control_table = DeltaTable.forPath(self.spark, self.control_table_path)

        control_table.alias("target").merge(
            new_row.alias("source"),
            "target.source_table = source.source_table"
        ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()


    def enable_cdf(self):
        self.spark.sql(f"""
          ALTER TABLE delta.`{self.landing_path}`
          SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
        """)


    def get_last_processed_version(self):
        df = self.spark.read.format("delta").load(self.control_table_path)
        result = df.filter(df.source_table == self.table_name).collect()
        if result:
            return result[0]['last_processed_version']
        else:
            return 0


    def get_latest_version(self):
        delta_table = DeltaTable.forPath(self.spark, self.landing_path)
        return delta_table.history(1).select("version").collect()[0][0]


    def get_new_data(self):
        last_processed_version = self.get_last_processed_version()
        delta_table = DeltaTable.forPath(self.spark, self.landing_path)

        cdf_enabled = is_cdf_enabled(delta_table)
        latest_version = self.get_latest_version()

        use_cdf = cdf_enabled and does_cdf_make_sense(delta_table, last_processed_version)

        if use_cdf:
            print(f"Using CDF from version {last_processed_version}")
            df = self.spark.read.format("delta") \
                .option("readChangeData", "true") \
                .option("startingVersion", str(last_processed_version)) \
                .option("endingVersion", str(latest_version)) \
                .load(self.landing_path) \
                .filter("_change_type = 'insert'")
        else:
            print("CDF not available — doing full load")
            if not cdf_enabled:
                self.enable_cdf()
            latest_version = self.get_latest_version()
            df = self.spark.read.format("delta") \
                 .option("versionAsOf", latest_version) \
                 .load(self.landing_path)

        return df, latest_version
